In [1]:
import graphBrowser as gB

In [2]:
graphName = '<airline_flight_network>'
gB.getTypesOfNodes(graphName)
gB.nodeTypes

['<https://ontologies.semanticarts.com/flights/Flight>',
 '<https://ontologies.semanticarts.com/flights/DomesticCountry>',
 '<https://ontologies.semanticarts.com/flights/Country>',
 '<https://ontologies.semanticarts.com/raw_data#Carrier>',
 '<https://ontologies.semanticarts.com/flights/ForeignCountry>',
 '<https://ontologies.semanticarts.com/flights/Airport>',
 '<https://ontologies.semanticarts.com/flights/City>']

In [3]:
colors = ['#58FAF4','#d1fa58','#fada58','#fa589c','#dc58fa','#6358fa','#f7fa58']
colorCode = {
    '<https://ontologies.semanticarts.com/flights/Country>': '#fa589c',
    '<https://ontologies.semanticarts.com/raw_data#Carrier>': '#d1fa58',
    '<https://ontologies.semanticarts.com/flights/City>': '#fada58',
    '<https://ontologies.semanticarts.com/flights/Airport>': '#58FAF4',
    '<https://ontologies.semanticarts.com/flights/ForeignCountry>': '#dc58fa',
    '<https://ontologies.semanticarts.com/flights/DomesticCountry>': '#6358fa',
    '<https://ontologies.semanticarts.com/flights/Flight>': '#f7fa58'
}
gB.assignColorsToNode(colorCode)
gB.nodeColors    

{'<https://ontologies.semanticarts.com/flights/Country>': '#fa589c',
 '<https://ontologies.semanticarts.com/raw_data#Carrier>': '#d1fa58',
 '<https://ontologies.semanticarts.com/flights/City>': '#fada58',
 '<https://ontologies.semanticarts.com/flights/Airport>': '#58FAF4',
 '<https://ontologies.semanticarts.com/flights/ForeignCountry>': '#dc58fa',
 '<https://ontologies.semanticarts.com/flights/DomesticCountry>': '#6358fa',
 '<https://ontologies.semanticarts.com/flights/Flight>': '#f7fa58'}

In [4]:


# Demo: construct creation function for airlines data
def createConstructQuery(origin,graphName):
    return '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix : <https://ontologies.semanticarts.com/raw_data#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix skos:    <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { 
  	?orig fl:hasRouteTo ?dest .
  	?orig rdfs:label ?origCode . 
  	?dest rdfs:label ?destCode . 
  	fl:hasRouteTo rdfs:label "hasRouteTo" . 
}  
WHERE { GRAPH '''+ graphName +''' {{
SELECT 
?orig ?dest ?origCode ?destCode ?miles 
    WHERE { 
        ?orig fl:terminalCode ?origCode .
        ?orig fl:hasRouteTo ?dest .
        ?dest fl:terminalCode ?destCode .
        << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
        FILTER (?miles < 400)
        FILTER (?origCode = \'''' + origin + '''\')
  }
      }}}
'''


construct = createConstructQuery('BOS',graphName)
elements = gB.createGraph(construct,'BOS')

In [ ]:

from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State

app = JupyterDash(__name__)

app.layout = gB.getAppLayout(elements)


##########CALLBACKS##########
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapNodeData')])
def displayTapNodeData(data):
    return gB.displayTapNodeData(data)


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapEdgeData')]
             )
def displayTapEdgeData(data):
    return gB.displayTapEdgeData(data)
    
@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return gB.update_layout(layout)

#expand node by neighbors
@app.callback(Output('cytoscape-update-layout', 'elements'),
              [Input('cytoscape-update-layout', 'tapNodeData')],
              [State('cytoscape-update-layout', 'elements')],
              [State('radio-option', 'value')]
             )
def generate_elements(data,e,options):
    return gB.generate_elements(data,e,options, graphName)
  
            
port = '8198'
host = 'localhost'

app.run_server(mode='inline',port = port, host = host)

print('App running on http://'+ host +':' + port +'/')

In [ ]:
elements